In [ ]:
# !pip install html5lib
# !pip install lxml
# !pip install beautifulSoup4

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [ ]:
page_no = 1
product_url = "https://search.shopping.naver.com/catalog/21098473707"
print(product_url)
url = f"https://search.shopping.naver.com/api/review?isNeedAggregation=N&nvMid=21098473707&page={page_no}&pageSize=20&sortType=QUALITY"
print(url)

In [ ]:
def find_review_page(page_no):
    url = f"https://search.shopping.naver.com/api/review"
    params = {
        "isNeedAggregation": "N",
        "nvMid": "21098473707",
        "page": page_no,
        "pageSize": 20,
        "sortType": "QUALITY"
    }

    headers = {
        "path": f"/api/review?isNeedAggregation=N&nvMid=21098473707&page={page_no}&pageSize=20&sortType=QUALITY",
        "referer":"https://search.shopping.naver.com/catalog/21098473707",
        "user-agent": "Mozilla/5.0 ",
        "sbth": "1bfd439d6b41f4624bea6c112e310b9c412871aba70f2f714eabcd8cf3c8c900a2693a53c38128be910a712072424a2a"
    }
    response = requests.get(url, params=params, headers=headers)
    return response

In [ ]:
page_no = 1
response = find_review_page(page_no)
print(response)
res_json = response.json()
res_json["totalCount"]

In [ ]:
pd.DataFrame(res_json["reviews"]).shape

In [ ]:
review_list = []
for page_no in range(1, 6):
    response = find_review_page(page_no)
    review_list.append(pd.DataFrame(response.json()["reviews"]))

In [ ]:
df_review = pd.concat(review_list, ignore_index=True)
df_review.shape

In [ ]:
df_review.columns

In [ ]:
df_review[["id", "content", "starScore", "qualityScore", "rankScore"]].head()

In [ ]:
df_review[["id", "content"]].tail()

In [ ]:
df_review[["id", "content"]].drop_duplicates()

In [ ]:
reviews = " ".join(df_review["title"].tolist())
reviews[:100]